In [2]:
import requests
import pdfplumber
from io import BytesIO
import pandas as pd

# Step 1: Fetch the PDF from the URL
pdf_url = "https://budget.finance.go.ug/sites/default/files/Sector%20Spending%20Agency%20Budgets%20and%20Performance/Ministry%20of%20Education%20and%20Sports_3.pdf"
response = requests.get(pdf_url)
pdf_file = BytesIO(response.content)

# Step 2: Extract tables from the PDF
with pdfplumber.open(pdf_file) as pdf:
    all_tables = []
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            if table:  # Ensure the table is not empty
                df = pd.DataFrame(table[1:], columns=table[0])  # Skip header row
                all_tables.append(df)

# Step 3: Combine all tables into a single DataFrame
if all_tables:
    combined_df = pd.concat(all_tables, ignore_index=True)
    print(combined_df.head())  # Display the first few rows
else:
    print("No tables found in the PDF.")


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [4]:
import pdfplumber
import pandas as pd

# Step: Extract and clean tables from the loaded PDF
all_tables = []

with pdfplumber.open(pdf_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        tables = page.extract_tables()
        for table_num, table in enumerate(tables):
            if table:
                try:
                    df = pd.DataFrame(table[1:], columns=table[0])
                    
                    # Check for duplicate columns
                    if not df.columns.is_unique:
                        # Auto-rename duplicate columns
                        df.columns = pd.io.parsers.ParserBase({'names': df.columns})._maybe_dedup_names(df.columns)
                        print(f"🔄 Renamed duplicate columns on page {page_num+1}, table {table_num+1}")
                    
                    all_tables.append(df)

                except Exception as e:
                    print(f"⚠️ Error processing table on page {page_num+1}, table {table_num+1}: {e}")

# Step: Combine all tables into a single DataFrame
if all_tables:
    combined_df = pd.concat(all_tables, ignore_index=True)
    print("✅ Combined DataFrame preview:")
    print(combined_df.head())
else:
    print("❌ No usable tables found in the PDF.")


⚠️ Error processing table on page 1, table 1: module 'pandas.io.parsers' has no attribute 'ParserBase'
⚠️ Error processing table on page 2, table 1: module 'pandas.io.parsers' has no attribute 'ParserBase'
⚠️ Error processing table on page 3, table 1: module 'pandas.io.parsers' has no attribute 'ParserBase'
⚠️ Error processing table on page 4, table 1: module 'pandas.io.parsers' has no attribute 'ParserBase'
⚠️ Error processing table on page 5, table 1: module 'pandas.io.parsers' has no attribute 'ParserBase'
⚠️ Error processing table on page 6, table 1: module 'pandas.io.parsers' has no attribute 'ParserBase'
⚠️ Error processing table on page 7, table 1: module 'pandas.io.parsers' has no attribute 'ParserBase'
⚠️ Error processing table on page 8, table 1: module 'pandas.io.parsers' has no attribute 'ParserBase'
⚠️ Error processing table on page 9, table 1: module 'pandas.io.parsers' has no attribute 'ParserBase'
⚠️ Error processing table on page 10, table 1: module 'pandas.io.parsers'

In [6]:
import pdfplumber
import pandas as pd

# Function to rename duplicate columns safely
def deduplicate_columns(cols):
    seen = {}
    new_cols = []
    for col in cols:
        if col in seen:
            seen[col] += 1
            new_cols.append(f"{col}.{seen[col]}")
        else:
            seen[col] = 0
            new_cols.append(col)
    return new_cols

# Extract tables from the already-loaded PDF file
all_tables = []

with pdfplumber.open(pdf_file) as pdf:
    for page_num, page in enumerate(pdf.pages):
        tables = page.extract_tables()
        for table_num, table in enumerate(tables):
            if table:
                try:
                    df = pd.DataFrame(table[1:], columns=table[0])

                    if not df.columns.is_unique:
                        df.columns = deduplicate_columns(df.columns)
                        print(f"🔄 Renamed duplicate columns on page {page_num+1}, table {table_num+1}")
                    
                    all_tables.append(df)

                except Exception as e:
                    print(f"⚠️ Error processing table on page {page_num+1}, table {table_num+1}: {e}")

# Combine all tables
if all_tables:
    combined_df = pd.concat(all_tables, ignore_index=True)
    print("✅ Combined DataFrame preview:")
    print(combined_df.head())
else:
    print("❌ No usable tables found in the PDF.")


🔄 Renamed duplicate columns on page 1, table 1
🔄 Renamed duplicate columns on page 2, table 1
🔄 Renamed duplicate columns on page 3, table 1
🔄 Renamed duplicate columns on page 4, table 1
🔄 Renamed duplicate columns on page 5, table 1
🔄 Renamed duplicate columns on page 6, table 1
🔄 Renamed duplicate columns on page 7, table 1
🔄 Renamed duplicate columns on page 8, table 1
🔄 Renamed duplicate columns on page 9, table 1
🔄 Renamed duplicate columns on page 10, table 1
🔄 Renamed duplicate columns on page 11, table 1
🔄 Renamed duplicate columns on page 12, table 1
🔄 Renamed duplicate columns on page 13, table 1
🔄 Renamed duplicate columns on page 14, table 1
🔄 Renamed duplicate columns on page 15, table 1
🔄 Renamed duplicate columns on page 16, table 1
🔄 Renamed duplicate columns on page 17, table 1
🔄 Renamed duplicate columns on page 18, table 1
🔄 Renamed duplicate columns on page 19, table 1
🔄 Renamed duplicate columns on page 20, table 1
🔄 Renamed duplicate columns on page 21, table 1
🔄

In [8]:

df.to_csv("education_vote013_extracted.csv", index=False)


In [10]:
df.to_excel("education_vote013_extracted.xlsx", index=False)


In [12]:
df["Performance (%)"] = (df["Actual"] / df["Approved"]) * 100
df["Deviation (UGX)"] = df["Approved"] - df["Actual"]
df["Execution Status"] = df["Performance (%)"].apply(lambda x: "Fully Spent" if x >= 95 else "Underutilized")


KeyError: 'Actual'